In [2]:
import numpy as np
import pandas as pd
import time
from util import fast_mapk,data_frame_to_lib_ffm
from sklearn.preprocessing import LabelEncoder
import time
import pickle

In [3]:
def add(a, b):
    return str(a) + str(b)

vfunc = np.vectorize(add)

#output_frame[column] = vfunc(str(column) + ':', output_frame[column])

In [4]:
#convert the data frame to lib ffm format
def data_frame_to_lib_ffm1(frame,key,field_list): #no le currenlty
    new_frame=pd.DataFrame()
    
    for column_name in frame.columns:
        if column_name in field_list:
            if column_name!=key:
                #col=[str(field_list[column_name])+':'+str(row)+':1' for row in frame[column_name]]  
                col= vfunc(str(frame[column_name]) , 1)
            else:
                col=frame[column_name]
            new_frame[column_name]=col
    return new_frame

In [5]:
import util
reload(util)

<module 'util' from 'util.pyc'>

In [6]:
#functions
def load_data_from_pickle():
    pkl_file = open('/datadrive/le.pickle', 'rb')
    le = pickle.load(pkl_file)

    pkl_file = open('/datadrive/full_train_features_dict.pickle', 'rb')
    full_train_features_dict = pickle.load(pkl_file)

    pkl_file = open('/datadrive/full_test_features_dict.pickle', 'rb')
    full_test_features_dict = pickle.load(pkl_file)

    return full_train_features_dict,full_test_features_dict

def generate_final_data_Frames(train_fields_dict,test_fields_dict):
    # generate final data frames
    t = time.time()
    final_train_frame = pd.DataFrame()
    final_test_frame = pd.DataFrame()

    for name in train_fields_dict:
        final_train_frame[name]=train_fields_dict[name]

    for name in test_fields_dict:
        final_test_frame[name]=test_fields_dict[name]

    print time.time() - t

    return final_train_frame,final_test_frame

def split(train_frame,test_frame,proportion=[0.3,0.4],split_type='small'):
    #start with small split
    t=time.time()
    display_ids=train_frame.display_id
    uniques = train_frame.display_id.unique()
    rands = np.random.rand(len(uniques))
    msk_train = rands < proportion[0]
    print sum(msk_train)
    msk_test =( rands < proportion[1]) & (rands >proportion[0])
    
    train = train_frame[train_frame.display_id.isin(uniques[msk_train])]
    print len(train)
    test = train_frame[train_frame.display_id.isin(uniques[msk_test])]
    print 'split finished',time.time() - t
    return train,test

In [7]:
df_train = pd.read_csv("clicks_train.csv")  # 100,000
df_test = pd.read_csv("clicks_test.csv")  # 100,000
print 'data read from files'

train_display_ids=df_train.display_id
test_display_ids = df_test.display_id

train_fields_dict, test_fields_dict=load_data_from_pickle()
train_frame, test_frame=generate_final_data_Frames(train_fields_dict,test_fields_dict)
print 'pickles loaded'

train_frame['display_id']=train_display_ids
test_frame['display_id']=test_display_ids
train_frame['clicked']=df_train.clicked
test_frame['clicked']=0
#after this line, I should have 2 dataframes with all data, 80M and 30M accordingly, which should be split if desired

#split:




data read from files
6.73870491982
pickles loaded


In [8]:
#split
train_size=0.7
train_split_frame,val_split_frame=split(train_frame,test_frame,proportion=[train_size,min(train_size*1.3,1)])

11800771
60939634
split finished 48.3518118858


In [9]:
#final frame to csv

t=time.time()
#this is also slowish... bcz of the data_frame_to_lib_ffm function
fields_list=['clicked', 'train_users', 'train_ads', 'train_dest_docs']#'train_categories':3
fields_dict={'clicked':-1, 'train_users':0, 'train_ads':1,'train_dest_docs':2}

final_train_frame=train_split_frame[fields_list]
print 'final_train length',len(final_train_frame)
final_val_frame=val_split_frame[fields_list]

lib_train_frame=data_frame_to_lib_ffm(final_train_frame, 'clicked',fields_dict) #(frame, key, field_list)
lib_val_frame=data_frame_to_lib_ffm(final_val_frame, 'clicked',fields_dict)
print time.time()-t

t=time.time()
lib_train_frame[fields_list].to_csv('train_frame.csv', sep=' ', header=None,index=False)
lib_val_frame[fields_list].to_csv('val_frame.csv', sep=' ', header=None,index=False)
print time.time()-t

final_train length 60939634
394.008224964
131.762354851


In [20]:
final_val_frame

,clicked,train_users,train_ads,train_dest_docs
10,0,1090107,19836377,20363042
11,0,6075581,19836377,20363042
12,0,9636615,19836377,20363042
13,0,16974116,19836377,20363042
14,1,6412636,19836377,20363042
15,0,7318474,19836377,20363042
16,0,15981450,19836377,20363042
17,1,11145665,19836377,20363042
18,0,17279679,19836377,20363042
19,0,4292094,19836377,20363042


array([20363042, 20363042, 20363042, ..., 20438439, 20521287, 20351614])

In [30]:
train_fields_dict['train_ads']

array([19836377, 19836377, 19836377, ..., 20037859, 20327162, 19897137])

In [23]:
max(final_val_frame.train_ads),min(final_val_frame.train_ads)

(20327801, 19794961)

#0.3 of the data:

final_train length 26118921

149.206947088

59.9669141769

In [40]:
min(train_fields_dict['train_dest_docs'])

20339260

In [55]:
train_fields_dict['train_ads'][19600:19700]

array([19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931

In [43]:
max(test_fields_dict['test_ads'])

20339260

In [44]:
train_fields_dict['train_users']

array([15471843, 16673234, 19473428, ..., 17425626, 16265036, 11819820])

In [48]:
le['uuid'].inverse_transform(16673234)

'da19c8b40f20ca'

In [46]:
le

{'ad_id': LabelEncoder(),
 'dest_doc_id': LabelEncoder(),
 'docs_cats': LabelEncoder(),
 'document_id': LabelEncoder(),
 'geo_location': LabelEncoder(),
 'uuid': LabelEncoder()}

In [38]:
le['train_users'].transform([938164])

array([21625])

In [ ]:
938164

In [10]:
#train model
#sudo ../../libffm/ffm-train -l 0.05 -k 20 -t 10 -r 0.2 -s 12 -p val_frame.csv train_frame.csv /datadrive/model


In [11]:
#predict


In [12]:
#evaluate 1
#read output
output_list=pd.read_csv('/datadrive/output',header=None)

#print len(output_list),len(lib_val_frame)

test_set=val_split_frame[['display_id','train_ads','clicked']]#
test_set['prediction']=output_list[0]
#join to test

# sort test set
#test_set=test_set.sort(['display_id','prediction'],ascending=[True,False])
test_set['prediction'].fillna(0)
test_set=test_set.rename(columns = {'prediction':'likelihood'})
test_set.head()

/home/ubuntu/.local/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,display_id,train_ads,clicked,likelihood
10,3,19836377,0,0.342424
11,3,19836377,0,0.300943
12,3,19836377,0,0.307157
13,3,19836377,0,0.475541
14,3,19836377,1,0.312758


In [13]:
#evaluate 2
#calc mapk on validation set
util.fast_mapk(test_set) #columns: display id, likelihood,clicked

0.0511960983276
0.577627897263
1.38568210602
0.00704598426819
0.0171511173248
MAP: 0.483482026567


In [31]:
pkl_file = open('/datadrive/le.pickle', 'rb')
le = pickle.load(pkl_file)

In [14]:
#prepare for submission

In [15]:
reload(util)

<module 'util' from 'util.pyc'>